<a href="https://colab.research.google.com/github/PenelopeRosswillTMU/CIND820_Capstone/blob/main/CIND_820_Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
from collections import Counter

In [12]:
#uploading dataset
df_url = 'https://raw.githubusercontent.com/PenelopeRosswillTMU/CIND820_Capstone/main/dataset_cleaned.csv'
df = pd.read_csv(df_url)
df.drop(['q10 Attribute','q11 Attribute','q14 Attribute','q17 Attribute'],axis=1,inplace=True)
df.columns = ['id','q1','q2','q3','q4','q5','q6','q7','q8','q9','q10','q11','q12']
questions = ['id','Do you agree with this statement: “in general, the services of the Ville de Montréal meet my needs”?','Do you agree with this statement: “in general, I feel comfortable being myself in my daily activities in Montreal”?','In the past two years, have you experienced unfavorable or differential treatment because of your gender identity and expression or your sexual orientation? Check all that apply. Please note that some of these services are offered by parties other than the Ville de Montréal.','Give your level of agreement with each of the following statements regarding the Montreal reality. Please note that some of these services are not offered by the City of Montreal.','In the past two years, have you experienced any of the following behaviors because of your gender identity and expression or sexual orientation? Check all that apply.','If you did not file a complaint, why? Check all that apply.','How many years have you lived in Montreal?','What age group are you in?','How do you identify yourself?','Do you identify as a trans person, or have you had a trans background at some point in your life?','What is your sexual orientation?','Are you a member of a First Nation or Inuit Nation?']
question_num = ['id','q1','q2','q3','q4','q5','q6','q7','q8','q9','q10','q11','q12']
q_ref = pd.DataFrame({'question_num':question_num,'question':questions})

In [13]:
#removing rows with 'nan' values
df = df[df['q1'].notna()]
df = df[df['q11'].notna()]

print(df.shape)

(143932, 13)


In [14]:
#dropping "I don't know" answers from the output variable to keep it to yes/no
df = df[df['q10'] != 'Je prÃ©fÃ¨re ne pas rÃ©pondre']
print(sorted(Counter(df['q10']).items()))
df_yes = df[df['q10'] == 'Oui']
df_no = df[df['q10'] == 'Non']
print(df_yes.shape,df_no.shape)

[('Non', 87280), ('Oui', 46596)]
(46596, 13) (87280, 13)


In [8]:
!pip install -U deep-translator
from deep_translator import GoogleTranslator
import re

unique_ans_fr = df['q1'].append(df['q2']).append(df['q3']).append(df['q4']).append(df['q5']).append(df['q6']).append(df['q7']).append(df['q8']).append(df['q9']).append(df['q10']).append(df['q11']).append(df['q12']).unique()
unique_ans_en = []
for i in range(len(unique_ans_fr)):
  translator = GoogleTranslator(source='fr',target='en')
  translate = translator.translate(str(unique_ans_fr[i]))
  unique_ans_en.append(translate)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00


<ipython-input-8-c5dafc58e4be>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_ans_fr = df['q1'].append(df['q2']).append(df['q3']).append(df['q4']).append(df['q5']).append(df['q6']).append(df['q7']).append(df['q8']).append(df['q9']).append(df['q10']).append(df['q11']).append(df['q12']).unique()
<ipython-input-8-c5dafc58e4be>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_ans_fr = df['q1'].append(df['q2']).append(df['q3']).append(df['q4']).append(df['q5']).append(df['q6']).append(df['q7']).append(df['q8']).append(df['q9']).append(df['q10']).append(df['q11']).append(df['q12']).unique()
<ipython-input-8-c5dafc58e4be>:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  unique_ans_fr = df['q1'].append(df['q2']).

In [16]:
#creating table with questions in French and English
unique_key = pd.DataFrame()
unique_key['Questions - French'] = unique_ans_fr
unique_key['Questions - English'] = unique_ans_en

unique_key['Questions - English'] = unique_key['Questions - English'].replace(['Rather in agreement'],'Somewhat agree')
unique_key['Questions - English'] = unique_key['Questions - English'].replace(['Rather disagree'],'Somewhat disagree')
unique_key['Questions - English'] = unique_key['Questions - English'].replace(['Nope'],'NaN')
unique_key['Questions - English'] = unique_key['Questions - English'].replace(['Happy'],'Gay')


#removing the duplicate "Other"
unique_key = unique_key.drop(labels=60, axis=0)
unique_key

,Questions - French,Questions - English
0,PlutÃ´t en accord,Somewhat agree
1,PlutÃ´t en dÃ©saccord,Somewhat disagree
2,Tout Ã fait en accord,Totally agree
3,Tout Ã fait en dÃ©saccord,Totally disagree
4,Au travail,At work
...,...,...
57,Bisexuelle,Bisexual
58,Gaie,Gay
59,Asexuelle,Asexual
61,HÃ©tÃ©rosexuelle,heterosexual


In [17]:
#replacing values in the French dataset with the English translations
for i in range(len(df.columns)):
  for j in range(len(df)):
    for k in range(62):
      if df.iloc[j,i] == unique_key.iloc[k,0]:
        df.iloc[j,i] = unique_key.iloc[k,1]

In [19]:
#downloading the resulting new english dataset
from google.colab import files
df.to_csv('dataset_translated.csv', encoding = 'utf-8-sig')
files.download('dataset_translated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>